<a href="https://colab.research.google.com/github/ketrabusek/Uczenie_maszynowe/blob/use_case3/zadanie3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install pandas_ta
!pip install backtrader

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.5/419.5 kB 6.5 MB/s eta 0:00:00


In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
import pandas_ta as ta
import matplotlib.pyplot as plt

# Pobranie danych
ticker = 'MSFT'
start_date = '2024-01-01'
end_date = '2024-05-06'
data = yf.download(ticker, start=start_date, end=end_date)

# Obliczenie dodatkowych wskaźników technicznych
data['MACD'] = ta.macd(data['Close'], fast=12, slow=26, signal=9)['MACD_12_26_9']
data['RSI'] = ta.rsi(data['Close'], length=14)
data['SMA50'] = ta.sma(data['Close'], length=50)

# Przygotowanie danych
data.dropna(inplace=True)
X = data[['Open', 'High', 'Low', 'Volume', 'MACD', 'RSI', 'SMA50']]
y = np.where(data['Close'].shift(-1) > data['Close'], 1, -1)
X = X[:-1]
y = y[:-1]

# Podział na zbiór treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model Random Forest
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)
best_rf = grid_search.best_estimator_

# Ocena modelu
y_pred = best_rf.predict(X_test)
print(classification_report(y_test, y_pred))

# Symulacja inwestycji
initial_balance = 10000
balance = initial_balance
positions = []

for i in range(len(X_test)):
    if y_pred[i] == 1:  # sygnał kupna
        buy_price = X_test.iloc[i]['Open']
        shares = balance / buy_price
        balance -= shares * buy_price
        positions.append((shares, buy_price))
    elif y_pred[i] == -1 and positions:  # sygnał sprzedaży
        shares, buy_price = positions.pop()
        sell_price = X_test.iloc[i]['Open']
        balance += shares * sell_price

# Wartość końcowa portfela
final_balance = balance + sum(shares * X_test.iloc[-1]['Close'] for shares, _ in positions)
print(f"Initial balance: ${initial_balance:.2f}")
print(f"Final balance: ${final_balance:.2f}")

# Wizualizacja
data['Signal'] = 0
data.loc[X_test.index, 'Signal'] = y_pred

plt.figure(figsize=(14, 7))
plt.plot(data.index, data['Close'], label='Close Price')
plt.plot(data[data['Signal'] == 1].index, data['Close'][data['Signal'] == 1], '^', markersize=10, color='g', label='Buy Signal')
plt.plot(data[data['Signal'] == -1].index, data['Close'][data['Signal'] == -1], 'v', markersize=10, color='r', label='Sell Signal')
plt.title('Microsoft (MSFT) Stock Price with Buy and Sell Signals')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.grid(True)
plt.show()


[*********************100%%**********************]  1 of 1 completed
